# ファインチューニング

In [13]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from openai import OpenAI

# 環境変数の取得
load_dotenv()
# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# モデル名
model_name = "gpt-3.5-turbo-1106"

## 学習データのチェック

https://cookbook.openai.com/examples/chat_finetuning_data_prep

In [7]:
import json
import tiktoken # for token counting
import numpy as np
from collections import defaultdict

In [8]:
data_path = "training.jsonl"

# Load the dataset
with open(data_path, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

Num examples: 10
First example:
{'role': 'user', 'content': 'どさ？'}
{'role': 'assistant', 'content': 'どこへ？'}


In [9]:
# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue
        
    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue
        
    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1
        
        if any(k not in ("role", "content", "name", "function_call") for k in message):
            format_errors["message_unrecognized_key"] += 1
        
        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1
            
        content = message.get("content", None)
        function_call = message.get("function_call", None)
        
        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1
    
    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

No errors found


## 学習データの作成とアップロード

In [10]:
# 学習データのアップロード
client.files.create(
  file=open("training.jsonl", "rb"),
  purpose="fine-tune"
)

FileObject(id='file-7WHeNOxALRxBQPrk7PKPHwPE', bytes=1132, created_at=1704612470, filename='training.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

## ファインチューニングの実行

In [11]:
# ファインチューニングジョブの開始
client.fine_tuning.jobs.create(
  training_file = "file-7WHeNOxALRxBQPrk7PKPHwPE", 
  model = model_name
)

FineTuningJob(id='ftjob-RUV9KV8LRt0a4kMaanI2laLF', created_at=1704612536, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-1106', object='fine_tuning.job', organization_id='org-xPjRpwlrimJbMHBKNimdi7UM', result_files=[], status='validating_files', trained_tokens=None, training_file='file-7WHeNOxALRxBQPrk7PKPHwPE', validation_file=None)

## 状況確認

In [14]:
# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-RUV9KV8LRt0a4kMaanI2laLF', created_at=1704612536, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=10, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-1106', object='fine_tuning.job', organization_id='org-xPjRpwlrimJbMHBKNimdi7UM', result_files=[], status='running', trained_tokens=None, training_file='file-7WHeNOxALRxBQPrk7PKPHwPE', validation_file=None)], object='list', has_more=False)

In [15]:
# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve("ftjob-RUV9KV8LRt0a4kMaanI2laLF")

FineTuningJob(id='ftjob-RUV9KV8LRt0a4kMaanI2laLF', created_at=1704612536, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=10, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-1106', object='fine_tuning.job', organization_id='org-xPjRpwlrimJbMHBKNimdi7UM', result_files=[], status='running', trained_tokens=None, training_file='file-7WHeNOxALRxBQPrk7PKPHwPE', validation_file=None)

In [16]:
# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-RUV9KV8LRt0a4kMaanI2laLF', created_at=1704612536, error=None, fine_tuned_model='ft:gpt-3.5-turbo-1106:personal::8eHyTOMW', finished_at=1704612904, hyperparameters=Hyperparameters(n_epochs=10, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-1106', object='fine_tuning.job', organization_id='org-xPjRpwlrimJbMHBKNimdi7UM', result_files=['file-5IjLsFd8idgDo0V6EzbNp8Si'], status='succeeded', trained_tokens=1970, training_file='file-7WHeNOxALRxBQPrk7PKPHwPE', validation_file=None)], object='list', has_more=False)

In [17]:
# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve("ftjob-RUV9KV8LRt0a4kMaanI2laLF")

FineTuningJob(id='ftjob-RUV9KV8LRt0a4kMaanI2laLF', created_at=1704612536, error=None, fine_tuned_model='ft:gpt-3.5-turbo-1106:personal::8eHyTOMW', finished_at=1704612904, hyperparameters=Hyperparameters(n_epochs=10, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-1106', object='fine_tuning.job', organization_id='org-xPjRpwlrimJbMHBKNimdi7UM', result_files=['file-5IjLsFd8idgDo0V6EzbNp8Si'], status='succeeded', trained_tokens=1970, training_file='file-7WHeNOxALRxBQPrk7PKPHwPE', validation_file=None)

## ファインチューニング済モデルの使用

In [22]:
# プロンプトの設定
content = "「かちゃくちゃねぇ」の意味は？"

# APIへリクエスト
response = client.chat.completions.create(
  model="ft:gpt-3.5-turbo-1106:personal::8eHyTOMW",
  messages=[
    {"role": "user", "content": content}
  ]
)

# 言語モデルからの回答を表示
print(response.choices[0].message.content.strip())

イライラする/イラつく
